### 2021/2/22
### 使用资料:/mydown/era5-model
### 本代码使用逐小时era5模式层数据来计算季风爆发前夕地表位涡的制造
### 只计算爆发前30天，不计算爆发后

In [1]:
import os
import numpy as np
import numpy.ma as ma
import Ngl, Nio
from netCDF4 import Dataset
import json
import sys
import math
sys.path.append("/mycode/module/")
from module_writenc import *
from module_sun import *
from attribute import *
with open("/data/onsetdate.json",'r') as load_f:
    a = json.load(load_f)

year = np.array(list(a.keys()))
day = np.array(list(a.values()))
year = year.astype(int)
day = day.astype(int)

* 获取基础信息

In [2]:
f0 = Nio.open_file("/mydown/era5-model/1995_modellev.nc")
lon = f0.variables["longitude"][:]
lat = f0.variables["latitude"][::-1]
level = f0.variables["level"][:]
disy,disx,location = cal_xydistance(lat,lon)

In [3]:
path_model = "/mydown/era5-model/"
path_ps = "/mydown/era5-ps/"

#### 定义一个函数使得ps和model数据时间上统一

In [4]:
def unanimous(year):
    path = "/mydown/era5-ps/"
    f1 = Nio.open_file(path+"ps."+str(year)+"-03.nc")
    f2 = Nio.open_file(path+"ps."+str(year)+"-04.nc")
    f3 = Nio.open_file(path+"ps."+str(year)+"-05.nc")
    f4 = Nio.open_file(path+"ps."+str(year)+"-06.nc")
    ps1 = f1.variables["sp"][:,30:151,30:121]
    ps1_1 = f1.variables["sp"]
    ps1 = ps1[:,::-1,:]*(ps1_1.scale_factor[0]) + (ps1_1.add_offset[0])
    ps2 = f2.variables["sp"][:,30:151,30:121]
    ps2_1 = f2.variables["sp"]
    ps2 = ps2[:,::-1,:]*(ps2_1.scale_factor[0]) + (ps2_1.add_offset[0])
    ps3 = f3.variables["sp"][:,30:151,30:121]
    ps3_1 = f3.variables["sp"]
    ps3 = ps3[:,::-1,:]*(ps3_1.scale_factor[0]) + (ps3_1.add_offset[0])
    ps4 = f4.variables["sp"][0:24,30:151,30:121]
    ps4_1 = f4.variables["sp"]
    ps4 = ps4[:,::-1,:]*(ps4_1.scale_factor[0]) + (ps4_1.add_offset[0])
    ps  = ma.zeros((ps1.shape[0]+ps2.shape[0]+ps3.shape[0]+ps4.shape[0],121,91))
    ps[0:ps1.shape[0],:,:] = ps1
    ps[ps1.shape[0]:(ps1.shape[0]+ps2.shape[0]),:,:] = ps2
    ps[(ps1.shape[0]+ps2.shape[0]):(ps1.shape[0]+ps2.shape[0]+ps3.shape[0]),:,:] = ps3
    ps[(ps1.shape[0]+ps2.shape[0]+ps3.shape[0]):,:,:] = ps4
    
    return ps

##### 把读取出来的数据反转再去short

def second_deal(var1):
    #var1带标签变量 var2纯数据
    var = var1[:,::-1,:]  #反转
    scale_factor = var1.scale_factor[0]
    add_offset = var1.add_offset[0]
    v2  =  var*scale_factor + add_offset
    return v2

def pv_create(u,v,tem,ps,location,disx):
    #这里使用逐小时计算整天的，故一次传入24个时次
    
    #三维涡度
    vz = (v[:,0,:,:]-v[:,1,:,:])/20
    uz = (u[:,0,:,:]-u[:,1,:,:])/20
    uy = ma.zeros((24,2,121,91))
    vx = ma.zeros((24,2,121,91))
    for t in range(0,u.shape[0]):
        for z in range(0,u.shape[1]):
            uy[t,z,:,:] = -1*np.gradient(u[t,z,:,:],location,axis=0)
            for latt in range(0,u.shape[2]):
                vx[t,z,latt,:] = np.gradient(v[t,z,latt,:],disx[latt],axis=0)
    vorticity = -vz+uz+vx[:,1,:,:]-uy[:,1,:,:]
    
    #计算theta
    theta = tem[:,1,:,:]*pow((100000/ps),0.286)
    #计算
    J_dynamic1 = -1*np.gradient(-vz,np.arange(1,25,1),axis=0)*theta
    J_dynamic2 = -1*np.gradient(uz,np.arange(1,25,1),axis=0)*theta
    J_dynamic3 = -1*np.gradient((vx[:,1,:,:]-uy[:,1,:,:]),np.arange(1,25,1),axis=0)*theta
    J_dynamic = -1*np.gradient(vorticity,np.arange(1,25,1),axis=0)*theta
    
    #计算当日的位涡制造
    J_dynamic_day1 = np.sum(J_dynamic1,axis=0)
    J_dynamic_day2 = np.sum(J_dynamic2,axis=0)
    J_dynamic_day3 = np.sum(J_dynamic3,axis=0)
    J_dynamic_day = np.sum(J_dynamic,axis=0)
    #J_day = J_dynamic_day + J_thermal_day
    
    return J_dynamic_day1,J_dynamic_day2,J_dynamic_day3,J_dynamic_day

### 主体程序

* 声明结果数组

In [5]:
J = ma.zeros((40,35,121,91))  #年日经纬度
J_dynamic = ma.zeros((40,35,121,91))
J_thermal = ma.zeros((40,35,121,91))
vora = ma.zeros((40,35,121,91))
vz= ma.zeros((40,35,121,91))
uz= ma.zeros((40,35,121,91))
vor1= ma.zeros((40,35,121,91))

##### 定位与切割,把爆发前30天+当天+爆发后4天的35*24小时数据截取出来

In [6]:
for yyyy in range(1980,2020):
    
    ps1 = unanimous(yyyy)
    f1 = Nio.open_file(path_model+str(yyyy)+"_modellev.nc")
    u1 = f1.variables["u"]
    v1 = f1.variables["v"]
    t1 = f1.variables["t"]
    u1 = second_deal(u1)
    v1 = second_deal(v1)
    t1 = second_deal(t1)

    day1 = int(a[str(yyyy)])
    if leap_year(yyyy): #需要刨去的
        day_front = day1 - 30 -61
    else:
        day_front = day1 - 30 -60
    hour1 = day_front * 24
    ps2 = ps1[hour1:hour1+840,:,:]
    u2  = u1[hour1:hour1+840,:,:,:]
    v2  = v1[hour1:hour1+840,:,:,:]
    t2  = t1[hour1:hour1+840,:,:,:]

    for dddd in range(0,35):
        #J[yyyy-1980,dddd,:,:],J_dynamic[yyyy-1980,dddd,:,:],J_thermal[yyyy-1980,dddd,:,:] = pv_create(u2[dddd*24:dddd*24+24,:,:,:],v2[dddd*24:dddd*24+24,:,:,:],t2[dddd*24:dddd*24+24,:,:,:],ps2[dddd*24:dddd*24+24,:,:], location,disx)
        J[yyyy-1980,dddd,:,:],J_dynamic[yyyy-1980,dddd,:,:],J_thermal[yyyy-1980,dddd,:,:],b = pv_create(u2[dddd*24:dddd*24+24,:,:,:],v2[dddd*24:dddd*24+24,:,:,:],t2[dddd*24:dddd*24+24,:,:,:],ps2[dddd*24:dddd*24+24,:,:], location,disx)
Jmean = np.sum(J,axis=0)/40
J_dynamicmean = np.sum(J_dynamic,axis=0)/40
J_thermalmean = np.sum(J_thermal,axis=0)/40

time = np.arange(0,35)
fout = create_nc_single('/data/','composite-era5pvcreate2',time,lon,lat)
a_w = {'longname': 'PVS', 'units': 'K m-1 s-1', 'valid_range': [-1000000000000000.0, 1000000000000000.0]}
a_v = {'longname': 'vorticity', 'units': 'm s-1', 'valid_range': [-1000000000000000.0, 1000000000000000.0]}
add_variables(fout,'j_vz',Jmean,a_w,0)
add_variables(fout,'j_uz',J_dynamicmean,a_w,0)
add_variables(fout,'j_horizontal',J_thermalmean,a_w,0)
add_variables(fout,'j_dynamic',b,a_w,0)

fout.close()